In [1]:
import talib
from datetime import datetime
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from functools import reduce
import quandl
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
from datetime import datetime
import selenium

In [2]:
from fake_useragent import UserAgent
ua = UserAgent()

ua.ie
# Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US);
ua.msie
# Mozilla/5.0 (compatible; MSIE 10.0; Macintosh; Intel Mac OS X 10_7_3; Trident/6.0)'
ua['Internet Explorer']
# Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.1; Trident/4.0; GTB7.4; InfoPath.2; SV1; .NET CLR 3.3.69573; WOW64; en-US)
ua.opera
# Opera/9.80 (X11; Linux i686; U; ru) Presto/2.8.131 Version/11.11
ua.chrome
# Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.2 (KHTML, like Gecko) Chrome/22.0.1216.0 Safari/537.2'
ua.google
# Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_4) AppleWebKit/537.13 (KHTML, like Gecko) Chrome/24.0.1290.1 Safari/537.13
ua['google chrome']
# Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11
ua.firefox
# Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16.0.1) Gecko/20121011 Firefox/16.0.1
ua.ff
# Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:15.0) Gecko/20100101 Firefox/15.0.1
ua.safari
# Mozilla/5.0 (iPad; CPU OS 6_0 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Version/6.0 Mobile/10A5355d Safari/8536.25

# and the best one, random via real world browser usage statistic
ua.random

'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_6; de-de) AppleWebKit/533.20.25 (KHTML, like Gecko) Version/5.0.4 Safari/533.20.27'

In [3]:
hzo = yf.Ticker("spy")

test = pd.DataFrame(hzo.history(start="1998-7-22", end="2021-09-30", interval="1d").values)

test.columns = ['Open','High','Low','Close','Volume','Dividends','Stock Splits']

test = test.drop('Dividends', axis = 1)

test = test.drop('Stock Splits', axis = 1)

Open = test['Open']
High = test["High"]
Low = test['Low']
Close = test['Close']
Volume = test['Volume']

test 

,Open,High,Low,Close,Volume
0,76.399958,76.820640,75.866410,76.502563,10417300.0
1,76.379403,76.564093,74.799281,74.983971,15110900.0
2,75.496991,75.579075,74.122080,74.922401,11540500.0
3,74.614592,75.517519,74.101565,75.455956,9600000.0
4,75.148153,75.291800,73.465425,74.183662,13693000.0
...,...,...,...,...,...
5832,439.850006,444.890015,439.600006,443.179993,76396000.0
5833,441.440002,444.670013,441.209991,443.910004,62025800.0
5834,442.809998,444.049988,441.899994,442.640015,61371100.0
5835,439.690002,440.040009,432.940002,433.720001,130436300.0


In [4]:
# ROC Input

ROC = talib.ROC(test['Close'],timeperiod=).values

ROC = pd.DataFrame(ROC[6:5836])

ROC.columns = ["ROC WEEKLY CHANGE"]

ROC

,ROC WEEKLY CHANGE
0,0.027411
1,-2.026847
2,-3.127542
3,-5.283542
4,-3.610069
...,...
5825,-1.924013
5826,-0.574775
5827,0.568647
5828,1.981386


In [5]:
#RSI Indicator

## Attach to 5:

RSI = talib.RSI(test["Close"], timeperiod = 5)

RSI = pd.DataFrame(RSI[5:5835].values)

RSI.columns = ["RSI CLOSE"]


RSI 

,RSI CLOSE
0,14.525130
1,37.931305
2,25.381848
3,23.512065
4,12.728476
...,...
5825,16.034847
5826,39.159498
5827,57.542440
5828,59.634435


In [6]:
#RSI Indicator

## Attach to 5:

#RSI2 = talib.RSI(test["Close"], timeperiod = 2)

#RSI2 = pd.DataFrame(RSI2[3:5835].values)

#RSI2.columns = ["RSI CLOSE 2"]


#RSI2

In [7]:
BOP = talib.BOP(Open, High, Low, Close)

BOP = pd.DataFrame(BOP[5:].values)

BOP.columns = ['BOP DAY PRIOR']

BOP = BOP[:-1]

BOP

,BOP DAY PRIOR
0,-0.633333
1,0.500000
2,-0.803922
3,-0.337079
4,-1.000000
...,...
5826,0.288216
5827,0.629486
5828,0.713869
5829,-0.079062


In [8]:
#df = pd.concat([ROC,RSI],axis=1)

df = pd.concat([ROC,RSI,BOP],axis = 1)

df

,ROC WEEKLY CHANGE,RSI CLOSE,BOP DAY PRIOR
0,0.027411,14.525130,-0.633333
1,-2.026847,37.931305,0.500000
2,-3.127542,25.381848,-0.803922
3,-5.283542,23.512065,-0.337079
4,-3.610069,12.728476,-1.000000
...,...,...,...
5826,-0.574775,39.159498,0.288216
5827,0.568647,57.542440,0.629486
5828,1.981386,59.634435,0.713869
5829,0.020754,53.863087,-0.079062


In [9]:
df.corr()['ROC WEEKLY CHANGE'].sort_values()

BOP DAY PRIOR        0.296774
RSI CLOSE            0.626457
ROC WEEKLY CHANGE    1.000000
Name: ROC WEEKLY CHANGE, dtype: float64